In [1]:
import tensorflow as tf 


In [57]:
class my_layer(tf.keras.layers.Layer) : 
  def __init__(self , units ,activation =None): 
    super(my_layer , self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)
  def build(self , input_shape) : 
    W_i = tf.random_normal_initializer( )
    self.w = tf.Variable(W_i(shape=(input_shape[-1] , self.units)) , dtype='float32' , trainable=True)
    b_i = tf.zeros_initializer()
    self.b = tf.Variable(b_i(shape=(self.units ,) ),dtype='float32' , trainable=True )

  def call(self , input) : 
    return self.activation(tf.matmul(input , self.w) + self.b)

In [54]:
class my_model(tf.keras.Model) : 
  def __init__(self, n_classes) :
    super(my_model , self).__init__()
    self.dense1 = my_layer(265 , 'relu')
    self.dense2 = my_layer(64 , 'relu')
    self.out = my_layer(n_classes, 'softmax')
  def call(self,input) : 
    X = self.dense1(input)
    X = self.dense2(X)
    return self.out(X) 

In [42]:
my_model(10)

In [43]:
train_loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_metrix = tf.keras.metrics.SparseCategoricalCrossentropy()
val_loss = tf.keras.losses.SparseCategoricalCrossentropy()
val_metrix  =tf.keras.metrics.SparseCategoricalCrossentropy()

In [58]:
def apply_step_gradient(model , optimizer , x ,y ,train_loss , train_metrix):
  with tf.GradientTape() as tape : 
    preds =model(x) 
    loss = train_loss( y ,preds )
  gradients = tape.gradient(loss , model.trainable_weights)
  optimizer.apply_gradients(zip(gradients , model.trainable_weights))
  return loss , preds  

In [66]:
def train_process(model ,optimizer , x , y , train_loss , train_metrix , verbose = True, num_epochs =20) : 
  train_losses = []
  val_loss = []
  for epoch in range(num_epochs) : 
    loss , logits  = apply_step_gradient(model , optimizer , x , y ,train_loss , train_metrix) 
    train_losses.append(loss)
    logits = tf.round(logits)
    logits = tf.cast(logits, 'int64')
    train_metrix.update_state( y,logits  )
    if verbose:
            print("Training loss for step %s: %.4f" % (int(epoch), float(loss)))
  return train_losses

In [47]:
(train , T_labels) , (test , labels ) = tf.keras.datasets.mnist.load_data()
# train /= 255 

In [48]:
train = train /255


In [49]:
train = tf.reshape(train , ( -1 ,28*28))


In [50]:
train = tf.cast(train , dtype='float32')

In [67]:
model = my_model(10) 
losses = train_process(model , optimizer , train , T_labels , train_loss , train_metrix )

Training loss for step 0: 2.2991
Training loss for step 1: 2.2785
Training loss for step 2: 2.2515
Training loss for step 3: 2.2201
Training loss for step 4: 2.1838
Training loss for step 5: 2.1410
Training loss for step 6: 2.0912
Training loss for step 7: 2.0345
Training loss for step 8: 1.9713
Training loss for step 9: 1.9019
Training loss for step 10: 1.8265
Training loss for step 11: 1.7452
Training loss for step 12: 1.6584
Training loss for step 13: 1.5678
Training loss for step 14: 1.4755
Training loss for step 15: 1.3829
Training loss for step 16: 1.2903
Training loss for step 17: 1.1988
Training loss for step 18: 1.1102
Training loss for step 19: 1.0263
